# 나이브베이즈

---

## Part1. 분류(Classification)

### 1. 분석 데이터 준비

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [3]:
data1 = pd.read_csv('../breast-cancer-wisconsin.csv')
data1.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [7]:
# 특성 데이터 분류
X = data1[data1.columns[1:-1]]
y = data1[['Class']]

In [10]:
# 훈련, 테스트 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)

In [12]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

### 2. 기본모델 적용 (나이브 베이즈)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.966796875

In [16]:
# 오차 행렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print('훈련데이터 오차행렬: \n', confusion_train)

훈련데이터 오차행렬: 
 [[319  14]
 [  3 176]]


In [17]:
# 레포트
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print('분류예측 레포트: \n', cfreport_train)

분류예측 레포트: 
               precision    recall  f1-score   support

           0       0.99      0.96      0.97       333
           1       0.93      0.98      0.95       179

    accuracy                           0.97       512
   macro avg       0.96      0.97      0.96       512
weighted avg       0.97      0.97      0.97       512



In [18]:
# 테스트 데이터
pred_test=model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9590643274853801

In [19]:
confusion_test=confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬:\n", confusion_test)

테스트데이터 오차행렬:
 [[106   5]
 [  2  58]]


In [20]:
from sklearn.metrics import classification_report
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



### 3. Grid Search

In [21]:
from sklearn.model_selection import GridSearchCV

In [25]:
param = {'var_smoothing' : [i for i in range(11)]}

In [26]:
grid_search = GridSearchCV(GaussianNB(), param, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

In [27]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'var_smoothing': 0}
Best Score: 0.9649
TestSet Score: 0.9591


### 4. Random Search

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [30]:
param = {'var_smoothing' : randint(low=0, high=20)}

In [31]:
random_search = RandomizedSearchCV(GaussianNB(), param, n_iter=100, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=GaussianNB(), n_iter=100,
                   param_distributions={'var_smoothing': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000212896F09D0>})

In [32]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'var_smoothing': 0}
Best Score: 0.9649
TestSet Score: 0.9591


---

## Part2. 회귀(Regression)

### 1. 분석 데이터 준비

In [33]:
data2 = pd.read_csv('../house_price.csv')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [37]:
# 특성 데이터 분류
X = data2[data2.columns[1:-1]]
y = data2[['house_value']]

In [38]:
# 훈련, 테스트 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [41]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

### 2. 기본모델 적용 (릿지모델)

In [42]:
from sklearn.linear_model import BayesianRidge

In [45]:
model = BayesianRidge()
model.fit(X_scaled_train, y_train)

# 훈련데이터
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.5455724466331763

In [46]:
# 테스트 데이터
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5626859871488648

In [47]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error

In [48]:
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 64340.34302948542
테스트 데이터 RMSE: 63220.68115643447


### 3. Grid Search

In [49]:
from sklearn.model_selection import GridSearchCV

In [50]:
param_grid={'alpha_1': [1e-06, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1, 2, 3, 4],
            'lambda_1': [1e-06, 1e-05, 1e-04, 1e-03, 1e-02, 1e-01, 1, 2, 3, 4]}

In [51]:
grid_search = GridSearchCV(BayesianRidge(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=BayesianRidge(),
             param_grid={'alpha_1': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1,
                                     2, 3, 4],
                         'lambda_1': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1,
                                      2, 3, 4]})

In [52]:
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Score: {:.4f}".format(grid_search.best_score_))
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter: {'alpha_1': 4, 'lambda_1': 1e-06}
Best Score: 0.5452
TestSet Score: 0.5627


### 4. Random Search

In [53]:
from sklearn.model_selection import RandomizedSearchCV

In [54]:
param_distribs = {'alpha_1': randint(low=1e-06, high=10),
            'lambda_1': randint(low=1e-06, high=10)}

In [55]:
random_search = RandomizedSearchCV(BayesianRidge(), param_distribs, n_iter=50, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=BayesianRidge(), n_iter=50,
                   param_distributions={'alpha_1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002128A8E4F10>,
                                        'lambda_1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002128A8FEF40>})

In [56]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'alpha_1': 7, 'lambda_1': 0}
Best Score: 0.5452
TestSet Score: 0.5627
